In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.utils import Sequence
from keras.layers import Conv2D, MaxPooling2D
from qkeras import *

from keras.utils import Sequence
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping

import os
import random

pi = 3.14159265359

maxval=1e9
minval=1e-9

import OptimizedDataGenerator as DG
from loss import *
from models import *
import utils

2024-08-28 20:36:18.829874: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 20:36:18.829960: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 20:36:18.831707: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-28 20:36:18.840313: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-28 20:36:20.114406: W tensorflow/compiler/tf2

Read from exisiting tf records

In [2]:
tfrecords_dir_train = "/depot/cms/users/kuang14/Smart_Pixel/tf_records/tfrecords_2t_dataset18_bs5000_train3"
tfrecords_dir_validation = "/depot/cms/users/kuang14/Smart_Pixel/tf_records/tfrecords_2t_dataset18_bs5000_val3"

In [3]:
validation_generator = DG.OptimizedDataGenerator(
    load_from_tfrecords_dir = tfrecords_dir_validation, 
    shuffle = True,
    seed = 13,
    quantize = True
)


In [4]:
training_generator =DG.OptimizedDataGenerator(
    load_from_tfrecords_dir = tfrecords_dir_validation, 
    shuffle = True,
    seed = 13,
    quantize = True
)

Training

In [5]:
input_shape = (13, 21, 2)
model = CreateModel(input_shape, n_filters=5, pool_size=3)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13, 21, 2)]       0         
                                                                 
 q_separable_conv2d (QSepar  (None, 11, 19, 5)         33        
 ableConv2D)                                                     
                                                                 
 q_activation (QActivation)  (None, 11, 19, 5)         0         
                                                                 
 q_conv2d (QConv2D)          (None, 11, 19, 5)         30        
                                                                 
 q_activation_1 (QActivatio  (None, 11, 19, 5)         0         
 n)                                                              
                                                                 
 average_pooling2d (Average  (None, 3, 6, 5)           0     

In [6]:
decay_steps = 90*training_generator.__len__()
alpha = 0.01
initial_learning_rate = 1e-3
warmup_target = 1e-1
warmup_steps = 10*training_generator.__len__()

lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    alpha=alpha,
    warmup_target = warmup_target,
    warmup_steps = warmup_steps
)


model.compile(
    optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-3),
    loss=custom_loss
)

# model.compile(
#     optimizer=tf.keras.optimizers.Nadam(learning_rate=lr_schedule),
#     loss=custom_loss
# )

In [7]:
fingerprint = '%08x' % random.randrange(16**8)
os.makedirs("trained_models", exist_ok=True)
base_dir = f'./trained_models/model-{fingerprint}-checkpoints'
os.makedirs(base_dir, exist_ok=True)  
checkpoint_filepath = base_dir + '/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'

In [8]:
checkpoint_filepath

'./trained_models/model-73e33061-checkpoints/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'

In [9]:
print(fingerprint)

73e33061


In [10]:
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, Callback

early_stopping_patience = 50

class CustomModelCheckpoint(ModelCheckpoint):
    def on_epoch_end(self, epoch, logs=None):
        super().on_epoch_end(epoch, logs)
        checkpoints = [f for f in os.listdir(base_dir) if f.startswith('weights')]
        if len(checkpoints) > 1:
            checkpoints.sort()
            for checkpoint in checkpoints[:-1]:
                os.remove(os.path.join(base_dir, checkpoint))

es = EarlyStopping(patience=early_stopping_patience, restore_best_weights=True)

mcp = CustomModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=True,
    save_freq='epoch',
    verbose=1
)

csv_logger = CSVLogger(f'{base_dir}/training_log.csv', append=True)

In [ ]:
model.fit(
    x=training_generator,
    validation_data=validation_generator,
    callbacks=[es, mcp, csv_logger],
    epochs=1000,
    shuffle=False,
    verbose=1
)

Epoch 1/1000
1028/1028 [==============================] - ETA: 0s - loss: 20404.6602
Epoch 1: val_loss improved from inf to 13933.39062, saving model to ./trained_models/model-73e33061-checkpoints/weights.01-t20404.66-v13933.39.hdf5
1028/1028 [==============================] - 325s 314ms/step - loss: 20404.6602 - val_loss: 13933.3906
Epoch 2/1000
1028/1028 [==============================] - ETA: 0s - loss: 12658.1104
Epoch 2: val_loss improved from 13933.39062 to 9955.94531, saving model to ./trained_models/model-73e33061-checkpoints/weights.02-t12658.11-v9955.95.hdf5
1028/1028 [==============================] - 332s 323ms/step - loss: 12658.1104 - val_loss: 9955.9453
Epoch 3/1000
1028/1028 [==============================] - ETA: 0s - loss: 9026.3477
Epoch 3: val_loss did not improve from 9955.94531
1028/1028 [==============================] - 340s 331ms/step - loss: 9026.3477 - val_loss: 11500.9854
Epoch 4/1000
1028/1028 [==============================] - ETA: 0s - loss: 6378.6309
Epo

In [ ]:
1

In [ ]:
print(fingerprint)

In [ ]:
# # clean up tf records
# utils.safe_remove_directory(tfrecords_dir_train)
# utils.safe_remove_directory(tfrecords_dir_validation)